# 第8章 統計的シミュレーション

## 選択問題及び部分記述問題 問13：メトロポリス・ヘイスティングス法

このノートブックでは、提供された問題（問13）の設定に基づき、メトロポリス・ヘイスティングス法（MH法）を用いた混合正規分布からのサンプリングをシミュレーションします。

### 問題の要約

*   **目標分布**: 混合正規分布
    $$\pi(x) = \frac{1}{4} N(0, 1) + \frac{3}{4} N(6, 1)$$
    
*   **アルゴリズム**: 酔歩連鎖によるメトロポリス・ヘイスティングス法
*   **目的**: 乱数 $x$ を 10000 個発生させる

### アルゴリズムの手順

1.  **Step 1**: 初期値 $x^{(0)} = 6$、$t \leftarrow 0$、ステップ幅パラメータ $\alpha = a$ を定める。
2.  **Step 2**: 提案分布から候補を生成する。
    $$\epsilon \sim U(-a, a)$$
    $$y = x^{(t)} + \epsilon$$
3.  **Step 3**: 採択確率 $\alpha(x^{(t)}, y)$ に基づいて更新する。
    $$u \sim U(0, 1)$$
    $$x^{(t+1)} = \begin{cases} y, & u \le \alpha(x^{(t)}, y) \\ x^{(t)}, & \text{それ以外} \end{cases}$$
    ここで、提案分布 $q(y|x)$ が対称（$U(-a, a)$）であるため、採択確率は以下のようになる。
    $$\alpha(x^{(t)}, y) = \min \left( 1, \frac{\pi(y)}{\pi(x^{(t)})} \right)$$
4.  **Step 4**: $t \leftarrow t + 1$
5.  **Step 5**: 
    *   $t \le 1000$ のとき、 $x^{(t)}$ は出力しない（バーンイン期間）。
    *   $1000 < t \le 11000$ のとき、 $x^{(t)}$ を出力する。
6.  **Step 6**: $t = 11000$ なら終了、それ以外なら Step 2 に戻る。

In [1]:
# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy.stats import norm
import matplotlib.font_manager as fm

# 日本語フォントの設定
try:
    font_path = '../fonts/ipaexg.ttf'
    font_prop = fm.FontProperties(fname=font_path)
    plt.rcParams['font.family'] = font_prop.get_name()
except:
    print("Font file not found, using default font.")

%matplotlib inline

In [2]:
# 目標分布の確率密度関数 pi(x)
def target_pdf(x):
    # 1/4 N(0, 1) + 3/4 N(6, 1)
    return 0.25 * norm.pdf(x, 0, 1) + 0.75 * norm.pdf(x, 6, 1)

# メトロポリス・ヘイスティングス法のサンプラー
def mh_sampler(a, n_iterations=11000, burn_in=1000, initial_x=6):
    samples = []
    trace = []
    
    x = initial_x
    current_pdf = target_pdf(x)
    
    # 採択回数のカウント（採択率計算用）
    accepted_count = 0
    
    for t in range(1, n_iterations + 1):
        # 提案分布: y = x + epsilon, epsilon ~ U(-a, a)
        epsilon = np.random.uniform(-a, a)
        y = x + epsilon
        
        # 提案された点の確率密度
        proposal_pdf = target_pdf(y)
        
        # 採択確率 alpha
        # 提案分布が対称なので q(x|y) = q(y|x) となり、比は pi(y)/pi(x) となる
        alpha = min(1, proposal_pdf / current_pdf)
        
        # 採択判定
        u = np.random.rand()
        if u <= alpha:
            x = y
            current_pdf = proposal_pdf
            accepted_count += 1
        
        # 記録
        trace.append(x)
        if t > burn_in:
            samples.append(x)
            
    acceptance_rate = accepted_count / n_iterations
    return np.array(samples), np.array(trace), acceptance_rate

### シミュレーションの実行

問題文にあるように、ステップ幅パラメータ $a$ を $0.1, 1, 6$ の3通りで実行し、結果を比較します。

In [3]:
a_values = [0.1, 1, 6]
results = {}

np.random.seed(42) # 再現性のため

for a in a_values:
    print(f"Running simulation for a = {a}...")
    samples, trace, acc_rate = mh_sampler(a, initial_x=6)
    results[a] = {'samples': samples, 'trace': trace, 'acc_rate': acc_rate}
    print(f"  Acceptance Rate: {acc_rate:.2%}")

Running simulation for a = 0.1...
  Acceptance Rate: 97.95%
Running simulation for a = 1...
  Acceptance Rate: 80.97%
Running simulation for a = 6...
  Acceptance Rate: 34.81%


### 結果の可視化

各 $a$ の値について、トレースプロット（サンプルの推移）とヒストグラム（真の分布との比較）を表示します。

In [4]:
# 真の分布のプロット用データ
x_grid = np.linspace(-4, 10, 1000)
y_true = target_pdf(x_grid)
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

plt.figure(figsize=(15, 12))

for i, a in enumerate(a_values):
    data = results[a]
    
    # トレースプロット
    ax = plt.subplot(3, 2, 2*i + 1)
    ax.plot(data['trace'], lw=0.5)
    ax.set_title(f'Trace Plot (a={a}, Accept={data["acc_rate"]:.2%})')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('x')
    ax.grid(alpha=0.3)

    inset_ax = inset_axes(ax, width="35%", height="35%", loc="upper right", borderpad=1)
    inset_ax.hist(data['samples'], bins=30, density=True, alpha=0.6, color='skyblue', orientation='horizontal')
    inset_ax.plot(y_true, x_grid, 'r-', lw=1)
    inset_ax.set_xticks([])
    inset_ax.set_yticks([])
    inset_ax.grid(alpha=0.2)
    
    # ヒストグラムと真の分布
    plt.subplot(3, 2, 2*i + 2)
    plt.hist(data['samples'], bins=50, density=True, alpha=0.6, color='skyblue', label='Samples', orientation='horizontal')
    plt.plot(y_true, x_grid, 'r-', lw=2, label='True PDF')
    plt.title(f'Histogram vs True PDF (a={a})')
    plt.xlabel('Density')
    plt.ylabel('x')
    plt.legend()
    plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

<Figure size 1500x1200 with 6 Axes>

### 問題の考察

#### [1] 式の導出
目標分布の確率密度関数 $\pi(x)$ は既に与えられています。
$$\pi(x) = \frac{1}{4} \phi(x) + \frac{3}{4} \phi(x-6)$$
ここで $\phi(x)$ は標準正規分布の確率密度関数です。

提案分布は**一様分布**で対称なので、

$$y \mid x^{(t)} \sim U\left(x^{(t)}-a, x^{(t)}+a
\right), \quad y = x^{(t)} + \epsilon,\ \epsilon\sim U(-a,a)$$

となります。
この確率密度関数は
$$q(y|x) = \frac{1}{2a} \mathbb{I}(x-a \le y \le x+a)$$
であり、$|y-x| \le a$ ならば $q(y|x) = 1/(2a)$、そうでなければ $0$ です。
対称性 $|y-x| = |x-y|$ より $q(y|x)=q(x|y)$ が成り立ち、メトロポリス法（Metropolis algorithm）の条件を満たします。

採択確率は

$$
\alpha\left(x^{(t)}, y
\right) = \min \left( 1, \frac{\pi(y)}{\pi(x^{(t)})} 
\right)
$$

で、比の中身を明示すると

$$
\frac{\pi(y)}{\pi(x^{(t)})}=\frac{\frac{1}{4}\phi(y) + \frac{3}{4}\phi(y-6)}{\frac{1}{4}\phi\left(x^{(t)}
\right) + \frac{3}{4}\phi\left(x^{(t)}-6
\right)}
$$

よって

$$
\alpha\left(x^{(t)}, y
\right)=\min \left( 1, \frac{\frac{1}{4}\phi(y) + \frac{3}{4}\phi(y-6)}{\frac{1}{4}\phi\left(x^{(t)}
\right) + \frac{3}{4}\phi\left(x^{(t)}-6
\right)} 
\right)
$$

となります。

#### [2] パラメータ $a$ と挙動の関係
シミュレーション結果（トレースプロットとヒストグラム）から、各 $a$ の挙動は以下のように解釈できます。

*   **$a=0.1$ の場合**: 
    *   ステップ幅が非常に小さいため、現在の位置から大きく動くことができません。
    *   初期値 $x=6$ の周辺（右側の山）にとどまり続けており、左側の山（$x=0$ 付近）へ移動できていません。
    *   **採択率**: 非常に高い（ほぼ 100% に近い）。提案された近傍の点は確率密度が近いため採択されやすいですが、混合（Mixing）が悪いです。

*   **$a=1$ の場合**:
    *   適度なステップ幅であり、 $x=6$ の山と $x=0$ の山の間を行き来できています。
    *   ヒストグラムも真の分布をよく再現しています。
    *   **採択率**: 適度な値（一般的に 20%~50% 程度が良いとされることが多いですが、この単純なケースでは 50-70% 程度でも良好です）。

*   **$a=6$ の場合**:
    *   ステップ幅が大きいため、現在の位置から遠く離れた、確率密度の低い領域を提案することが多くなります。
    *   その結果、棄却されることが多くなり、同じ値にとどまる期間（トレースプロットでの平坦な部分）が長くなります。
    *   **採択率**: 低い。

#### [3] バーンイン（Burn-in）について
**質問**: Step 5 で $x^{(1)}, ..., x^{(1000)}$ を出力に加えない理由を説明せよ。

**回答**: 
マルコフ連鎖モンテカルロ法（MCMC）では、連鎖の初期状態は初期値 $x^{(0)}$ の影響を強く受けており、目標分布 $\pi(x)$ からのサンプリングとはみなせません。十分なステップ数を経て連鎖が定常分布（目標分布）に収束するのを待つ必要があります。この収束までの期間を**バーンイン（Burn-in）**と呼び、この期間のサンプルを破棄することで、初期値依存性を排除し、より正確に目標分布に従うサンプルを得ることができます。

実際に推定したい量 $\mathbb{E}_{\pi}[f(X)]$ は、バーンイン後のサンプルで

$$
\hat{\mu}_f = \frac{1}{N}\sum_{t=B+1}^{B+N} f\left(x^{(t)}
\right)
$$

のように推定するため、$B$ ステップの破棄は初期値の偏りを減らすための処理になります。

#### [4] ARモデル（第7章）との関連

本節の「酔歩（ランダムウォーク）」による候補生成 $y = x^{(t)} + \epsilon$ は、**第7章のARモデル**と密接な関係があります。

第7章で扱った AR(1) モデル：
$$x_t = \phi x_{t-1} + \epsilon_t$$
（※自己回帰係数 $\phi$。本節の採択確率 $\alpha$ と区別しています）

ランダムウォークは、この自己回帰係数が $\phi = 1$ の場合に相当します。
*   **$\phi = 1$ (ランダムウォーク)**: **非定常**。過去の値の影響が減衰せず、分散が時間と共に拡大します。これにより、MCMCサンプラーは空間全体を自由に探索（Explore）できます。
*   **$|\phi| < 1$ (定常AR)**: **定常**。平均（通常0）へ戻ろうとする力が働き、分散が一定範囲に収まります。

MCMCの提案分布にランダムウォーク（単位根AR）を使う理由は、定常ARのように特定の場所に留まろうとせず、どこまでも移動できる性質が「未知の分布の探索」に適しているからです。その「拡散」を、目標分布 $\pi(x)$ に基づく採択確率で制御しています。


### インタラクティブな可視化 (Mixing & Diffusion)

以下のセルを実行すると、ステップ幅 $a$ (スライダー) を動かしながら、RW-MH法の挙動（混合の良さ、拡散の速さ）を視覚的に確認できるツールが表示されます。

*   **Trace + Running Mean**: トレースプロットとランニング平均（推定量が収束していく様子）。
*   **Histogram vs $\pi(x)$**: ヒストグラムと真の分布の比較。
*   **ACF**: 自己相関関数。混合が遅いとACFがなかなか減衰しません。
*   **MSD (Mean Squared Displacement)**: 平均二乗変位。拡散の速さを表します。

※ **注**: 本ツールでは、デモンストレーションのため提案分布に **正規分布** $y \sim N(x, a^2)$ を使用しています（本文中の $U(-a, a)$ とは異なります）。$a$ のスケール感が若干異なりますが、定性的な挙動（$a$ が小さすぎると拡散が遅く、$a$ が大きすぎると棄却が増える）は同様です。

In [ ]:
# --- RW-MH "mixing / diffusion" interactive viewer (a slider) --- 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.stats import norm 
 
try: 
    import ipywidgets as W 
    from IPython.display import display 
except Exception as e: 
    raise RuntimeError("ipywidgets が必要。Jupyterで ipywidgets を有効化してから実行して。") from e 
 
# ========= target π(x) = 1/4 N(0,1) + 3/4 N(6,1) ========= 
def pi_pdf(x): 
    return 0.25*norm.pdf(x, 0, 1) + 0.75*norm.pdf(x, 6, 1) 
 
def pi_log(x): 
    # log-sum-exp 安全版 
    a = np.log(0.25) + norm.logpdf(x, 0, 1) 
    b = np.log(0.75) + norm.logpdf(x, 6, 1) 
    m = np.maximum(a, b) 
    return m + np.log(np.exp(a-m) + np.exp(b-m)) 
 
# ========= RW-MH sampler ========= 
def mh_rw(a, n=12000, b=2000, x0=6.0, sd=0): 
    """ 
    a: proposal step scale (y = x + a*z) 
    n: total iters 
    b: burn-in 
    returns: x_post, acc_rate, stay_rate (fraction of repeated states) 
    """ 
    rng = np.random.default_rng(sd) 
    x = float(x0) 
    xs = np.empty(n, float) 
    acc = 0 
    stay = 0 
    lx = pi_log(x) 
    for t in range(n): 
        y = x + a*rng.standard_normal() 
        ly = pi_log(y) 
        if np.log(rng.random()) <= (ly - lx):  # accept 
            x, lx = y, ly 
            acc += 1 
        else: 
            stay += 1 
        xs[t] = x 
    return xs[b:], acc/n, stay/n 
 
# ========= diagnostics ========= 
def acf(x, L=400): 
    x = np.asarray(x) 
    n = len(x) 
    x = x - x.mean() 
    L = min(L, n-1) 
    c = np.correlate(x, x, mode="full")[n-1:n+L] 
    # unbiased autocov 
    d = np.arange(n, n-L-1, -1) 
    c = c/d 
    r = c/c[0] 
    return r  # lag 0..L 
 
def tau_int(r): 
    # ρ(h) が初めて非正になる直前までを足す（雑だが視覚教材には十分） 
    H = 0 
    for h in range(1, len(r)): 
        if r[h] <= 0: 
            H = h-1 
            break 
    else: 
        H = len(r)-1 
    t = 1.0 + 2.0*np.sum(r[1:H+1]) 
    return max(t, 1.0), H 
 
def msd(x, L=200): 
    # E[(X_{t+h}-X_t)^2] をラグhで見る（拡散の遅さが見える） 
    x = np.asarray(x) 
    n = len(x) 
    L = min(L, n//2) 
    out = np.empty(L+1) 
    out[0] = 0.0 
    for h in range(1, L+1): 
        d = x[h:] - x[:-h] 
        out[h] = np.mean(d*d) 
    return out 
 
# ========= plotting ========= 
def view(a=1.0, n=12000, b=2000, x0=6.0, seed=42, L=400): 
    xs, ar, sr = mh_rw(a, n=int(n), b=int(b), x0=float(x0), sd=int(seed)) 
    N = len(xs) 
    tru_m = 4.5        # E[X] for this mixture 
    tru_q = 0.25       # P(X<3) for this mixture (since components centered 0 and 6) 
 
    # functions for diagnostics 
    fx = xs 
    gx = (xs < 3.0).astype(float) 
 
    rx = acf(fx, L=L) 
    rg = acf(gx, L=L) 
    tx, Hx = tau_int(rx) 
    tg, Hg = tau_int(rg) 
    ne_x = N/tx 
    ne_g = N/tg 
 
    # mode switching 
    sw = np.sum(gx[1:] != gx[:-1])  # transitions count 
    p_hat = gx.mean() 
    m_hat = fx.mean() 
 
    # MSD 
    md = msd(fx, L=min(200, N//2)) 
 
    # plotting (4 panels) 
    fig = plt.figure(figsize=(12, 9)) 
 
    # (1) trace + running mean 
    ax1 = fig.add_subplot(2,2,1) 
    ax1.plot(xs, linewidth=0.7) 
    rm = np.cumsum(xs)/np.arange(1, N+1) 
    ax1.plot(rm, linewidth=1.5) 
    ax1.axhline(tru_m, linestyle="--", linewidth=1) 
    ax1.set_title(f"Trace & running mean  (a={a}, acc={ar*100:.1f}%, stay={sr*100:.1f}%)") 
    ax1.set_xlabel("t (post burn-in)") 
    ax1.set_ylabel("x") 
 
    # (2) histogram vs true pdf 
    ax2 = fig.add_subplot(2,2,2) 
    ax2.hist(xs, bins=60, density=True, alpha=0.5, orientation="horizontal") 
    yy = np.linspace(-4, 11, 600) 
    ax2.plot(pi_pdf(yy), yy, linewidth=2) 
    ax2.set_title(f"Histogram vs π(x)   mean={m_hat:.3f} (true {tru_m})") 
    ax2.set_xlabel("density") 
    ax2.set_ylabel("x") 
 
    # (3) ACF (x) and ACF (I[x<3]) 
    ax3 = fig.add_subplot(2,2,3) 
    ax3.plot(rx, label=f"ACF of x   tau≈{tx:.1f}, Neff≈{ne_x:.0f} (sum to lag {Hx})") 
    ax3.plot(rg, label=f"ACF of I[x<3]  tau≈{tg:.1f}, Neff≈{ne_g:.0f} (sum to lag {Hg})") 
    ax3.axhline(0, linewidth=1) 
    ax3.set_title("Autocorrelation (mixing = how fast ACF decays)") 
    ax3.set_xlabel("lag h") 
    ax3.set_ylabel("rho(h)") 
    ax3.legend(loc="upper right", fontsize=9) 
 
    # (4) diffusion view: MSD + mode-indicator summary 
    ax4 = fig.add_subplot(2,2,4) 
    ax4.plot(md, linewidth=2) 
    ax4.set_title(f"MSD vs lag (diffusion speed)\nP(X<3)={p_hat:.3f} (true {tru_q}), switches={sw}") 
    ax4.set_xlabel("lag h") 
    ax4.set_ylabel("E[(X_{t+h}-X_t)^2]") 
 
    plt.tight_layout() 
    display(fig)
    plt.close(fig)
 
# ========= UI ========= 
sl_a = W.FloatLogSlider(value=1.0, base=10, min=-1, max=1, step=0.05, description="a", readout_format=".3f") 
sl_n = W.IntSlider(value=12000, min=3000, max=30000, step=1000, description="n") 
sl_b = W.IntSlider(value=2000, min=0, max=10000, step=500, description="burn") 
sl_x0 = W.FloatSlider(value=6.0, min=-4, max=10, step=0.5, description="x0") 
sl_sd = W.IntText(value=42, description="seed") 
sl_L = W.IntSlider(value=400, min=50, max=1500, step=50, description="ACF_L") 
 
ui = W.VBox([ 
    W.HBox([sl_a, sl_n]), 
    W.HBox([sl_b, sl_x0, sl_sd]), 
    sl_L 
]) 
 
out = W.interactive_output(view, {"a":sl_a, "n":sl_n, "b":sl_b, "x0":sl_x0, "seed":sl_sd, "L":sl_L}) 
display(ui, out) 
